In [23]:
import pandas as pd
import pandas_ta as ta
import importlib
import os
import sys
import yfinance as yf

module_path = os.path.abspath(os.path.join("..", "src"))

if module_path not in sys.path:
    sys.path.append(module_path)

Input needed:

1. prices.csv
-date, asset, Close, Expected Return (from AR model), other components needed to run strategies

2. strategy_map.csv
- asset, strategy (e.g., Strategy05), ideal_proportion

Config Example:

config = {
    "starting_cash": 1000000,        # Initial capital
    "buy_pct": 0.05,               # Each buy = 5% of max cap
    "cash_floor_pct": 0.10,        # Minimum 10% cash must be held
    "cash_ceiling_pct": 0.30,      # If cash > 30%, consider buying gold
    "fee": 0.001,                  # Transaction fee (0.1%)
    "tp_pct": 0.10,                # Take-profit threshold (10%) 
    "sl_pct": 0.05                 # Stop-loss threshold (5%)
}

Notes: the tp, sl pct gonna make "dynamic" - different for each asset.

In [24]:
# Create some sample data to try now
config = {
    "starting_cash": 1000000,  # Initial capital
    "buy_pct": 1,  # Each buy = 5% of max cap
    "cash_floor_pct": 0.10,  # Minimum 10% cash must be held
    "tp_pct": 0.1,  # Take-profit threshold (10%)
    "sl_pct": 0.05,  # Stop-loss threshold (5%)
}

classname_to_filename = {
    "BollingerBandsBreakout": "bollinger_bands_breakout",
    "LarryWilliamsPriceAction": "larry_williams_price_action",
    "MACDBollingerBandsMeanReversion": "macd_bollinger_bands_mean_reversion",
    "MeanReversion": "mean_reversion",
    "MichaelHarrisPriceAction": "michael_harris_price_action",
    "Momentum": "momentum",
    "RSIDivergence": "rsi_divergence",
    "Scalping": "scalping",
    "VolumeSpikeReversal": "volume_spike_reversal",
}

#CHANGE FILE HERE
strategy_map = pd.read_csv("../data/experiments/asset_strategies_2_months_with_tpsl.csv")
strategy_map.head()

,Start,End,Duration,Exposure Time [%],Equity Final [$],Equity Peak [$],Return [%],Buy & Hold Return [%],Return (Ann.) [%],Volatility (Ann.) [%],...,Max. Trade Duration,Avg. Trade Duration,Profit Factor,Expectancy [%],SQN,Kelly Criterion,Strategy,Asset,Weight,Commissions [$]
0,2025-01-02 00:00:00,2025-02-28 00:00:00,57 days 00:00:00,0.000000,58778.750476,58778.750476,0.000000,-9.778786,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,BollingerBandsBreakout,AES,0.058779,NaN
1,2025-01-02 00:00:00,2025-02-28 00:00:00,57 days 00:00:00,5.128205,55067.499180,58778.750476,-6.313934,-9.778786,-34.388955,8.138026,...,1 days 00:00:00,1 days 00:00:00,0.0,-6.509081,NaN,NaN,LarryWilliamsPriceAction,AES,0.058779,27.376993
2,2025-01-02 00:00:00,2025-02-28 00:00:00,57 days 00:00:00,0.000000,58778.750476,58778.750476,0.000000,9.442870,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,MACDBollingerBandsMeanReversion,AES,0.058779,NaN
3,2025-01-02 00:00:00,2025-02-28 00:00:00,57 days 00:00:00,2.564103,58549.668271,58778.750476,-0.389736,5.363638,-2.491650,0.978759,...,0 days 00:00:00,0 days 00:00:00,0.0,-0.357466,NaN,NaN,MeanReversion,AES,0.058779,28.066472
4,2025-01-02 00:00:00,2025-02-28 00:00:00,57 days 00:00:00,7.692308,55705.398517,58861.914976,-5.228679,-9.778786,-29.319905,7.817601,...,4 days 00:00:00,4 days 00:00:00,0.0,-5.410194,NaN,NaN,MichaelHarrisPriceAction,AES,0.058779,27.388732


In [25]:
idx= strategy_map.groupby("Asset")["Return [%]"].idxmax()
best = strategy_map.loc[idx]
strategy_map_new = best[["Asset", "Weight", "Strategy", "Return [%]"]]
strategy_map_new = strategy_map_new.set_index("Asset")
strategy_map_new

,Weight,Strategy,Return [%]
Asset,,,
ADA-USD,0.016802,MACDBollingerBandsMeanReversion,0.000000
ADI,0.028764,Scalping,12.266128
AES,0.058779,Momentum,6.994123
AKAM,0.056386,BollingerBandsBreakout,0.000000
BLDR,0.042793,MichaelHarrisPriceAction,10.208296
CARR,0.029253,BollingerBandsBreakout,0.000000
CBRE,0.041216,Scalping,13.635765
CSGP,0.033277,BollingerBandsBreakout,0.000000
DECK,0.043717,LarryWilliamsPriceAction,6.126716


First function retrieves data from yf 

In [26]:
def get_asset_df(strategy_map, start, end):
    df_list = []
    for asset_name in strategy_map.index:
        # Retrieve yf finance data
        df = yf.Ticker(asset_name).history(start=start, end=end, actions=False)

        # Align date formatting
        df = df.reset_index()
        df["Date"] = pd.to_datetime(df["Date"], errors="coerce").dt.strftime("%Y-%m-%d")

        df["Asset"] = asset_name
        df_list.append(df.reset_index())

    df_all = pd.concat(df_list).set_index(["Date", "Asset"]).sort_index()
    return df_all

In [27]:
asset_df = get_asset_df(strategy_map=strategy_map_new, start="2023-01-01", end="2025-03-01")
asset_df.head()

index         Open         High          Low  \
Date       Asset                                                    
2023-01-01 ADA-USD       0     0.246414     0.249771     0.244245   
           ETH-USD       0  1196.713623  1203.475342  1192.885376   
           LINK-USD      0     5.568981     5.628169     5.517975   
           LTC-USD       0    70.002823    71.328911    69.234886   
           SOL-USD       0     9.961036    10.052801     9.721011   

                           Close      Volume  
Date       Asset                              
2023-01-01 ADA-USD      0.249771   113369989  
           ETH-USD   1200.964844  2399674550  
           LINK-USD     5.622443   109175362  
           LTC-USD     70.815659   344383885  
           SOL-USD      9.982173   194221164

Second function implements AR model to derive expected return 

In [28]:
def compute_returns(asset_df):
    asset_df["Return"] = asset_df.groupby("Asset")["Close"].pct_change()
    return asset_df


asset_df_with_returns = compute_returns(asset_df)

In [29]:
asset_df_with_returns.head(10)

index         Open         High          Low  \
Date       Asset                                                    
2023-01-01 ADA-USD       0     0.246414     0.249771     0.244245   
           ETH-USD       0  1196.713623  1203.475342  1192.885376   
           LINK-USD      0     5.568981     5.628169     5.517975   
           LTC-USD       0    70.002823    71.328911    69.234886   
           SOL-USD       0     9.961036    10.052801     9.721011   
           UNI-USD       0     0.000099     0.000100     0.000099   
           XRP-USD       0     0.339923     0.339923     0.336332   
2023-01-02 ADA-USD       1     0.249778     0.255767     0.247368   
           ETH-USD       1  1201.103271  1219.860596  1195.214966   
           LINK-USD      1     5.622763     5.737175     5.567815   

                           Close      Volume    Return  
Date       Asset                                        
2023-01-01 ADA-USD      0.249771   113369989       NaN  
           ETH-USD   1200.964844  2399674550       NaN  
           LINK-USD     5.622443   109175362       NaN  
           LTC-USD     70.815659   344383885       NaN  
           SOL-USD      9.982173   194221164       NaN  
           UNI-USD      0.000100           5       NaN  
           XRP-USD      0.338763   290828851       NaN  
2023-01-02 ADA-USD      0.253828   159328803  0.016243  
           ETH-USD   1214.656616  3765758498  0.011401  
           LINK-USD     5.687627   179768004  0.011593

In [30]:
def generate_expected_returns(data, ar_results):
    """ Calculate expected returns using fitted AR models """
    new_data = data.reset_index().merge(ar_results, on="Asset", how="left").set_index(["Date", "Asset"])
    new_data["Expected Return"] = new_data["const"] + new_data["Return"] * new_data["ar.L1"]
    return new_data

In [31]:
ar_model_info = pd.read_csv("../data/processed/ar_model.csv")
ar_model_info

,Asset,Best Model,BIC,const,ar.L1,sigma2
0,CZR,"(1, 0, 0)",-2137.099847,-0.001004,-0.037306,0.000487
1,INTC,"(1, 0, 0)",-1898.767394,-0.001165,0.053588,0.000827
2,MHK,"(1, 0, 0)",-2237.419365,0.000454,0.064773,0.000389
3,BLDR,"(1, 0, 0)",-2146.278473,-0.000304,-0.037379,0.000477
4,URI,"(1, 0, 0)",-2284.821016,0.000504,-0.071571,0.000350
5,ON,"(1, 0, 0)",-2053.295520,-0.001143,-0.153363,0.000586
6,NCLH,"(1, 0, 0)",-2008.360786,0.000543,-0.014170,0.000648
7,ALB,"(1, 0, 0)",-1908.997544,-0.000997,-0.151407,0.000808
8,VST,"(1, 0, 0)",-1739.133780,0.003166,-0.093050,0.001179
9,AVAX-USD,"(1, 0, 0)",-1402.471749,-0.000203,0.003612,0.002473


In [32]:
# Apply AR model
asset_df_with_AR = generate_expected_returns(asset_df_with_returns, ar_model_info)
asset_df_with_AR = asset_df_with_AR.drop(['Best Model','BIC', 'const','ar.L1', 'sigma2'], axis=1)
asset_df_with_AR

index         Open         High          Low  \
Date       Asset                                                    
2023-01-01 ADA-USD       0     0.246414     0.249771     0.244245   
           ETH-USD       0  1196.713623  1203.475342  1192.885376   
           LINK-USD      0     5.568981     5.628169     5.517975   
           LTC-USD       0    70.002823    71.328911    69.234886   
           SOL-USD       0     9.961036    10.052801     9.721011   
...                    ...          ...          ...          ...   
2025-02-28 SOL-USD     789   137.620773   148.183838   125.742218   
           STZ         540   174.970001   176.699997   173.029999   
           TSLA        540   279.500000   293.880005   273.600006   
           UNI-USD     789     0.000170     0.000170     0.000158   
           XRP-USD     789     2.198640     2.198640     1.957408   

                           Close      Volume    Return  Expected Return  
Date       Asset                                                         
2023-01-01 ADA-USD      0.249771   113369989       NaN              NaN  
           ETH-USD   1200.964844  2399674550       NaN              NaN  
           LINK-USD     5.622443   109175362       NaN              NaN  
           LTC-USD     70.815659   344383885       NaN              NaN  
           SOL-USD      9.982173   194221164       NaN              NaN  
...                          ...         ...       ...              ...  
2025-02-28 SOL-USD    148.030014  7544673033  0.075637        -0.004608  
           STZ        175.500000     2913300  0.013045              NaN  
           TSLA       292.980011   115697000  0.039120              NaN  
           UNI-USD      0.000170           3  0.000000              NaN  
           XRP-USD      2.145595  8544047699 -0.024130              NaN  

[17973 rows x 8 columns]

In [33]:
def calculate_technical_indicators(df):
    df = df.copy()

    # Core indicators
    df["EMA_50"] = ta.ema(df["Close"], length=50)
    df["EMA_200"] = ta.ema(df["Close"], length=200)
    df["RSI"] = ta.rsi(df["Close"], length=14)
    df["ATR"] = ta.atr(df["High"], df["Low"], df["Close"], length=7)

    # Bollinger Bands
    bbands = ta.bbands(df["Close"], length=20)
    bbands = bbands.rename(
        columns={
            "BBU_20_2.0": "Upper_Band",
            "BBM_20_2.0": "Middle_Band",
            "BBL_20_2.0": "Lower_Band",
            "BBB_20_2.0": "Band_Width",
            "BBP_20_2.0": "Percent_B",
        }
    )

    # MACD
    macd = ta.macd(df["Close"])
    macd = macd.rename(
        columns={
            "MACD_12_26_9": "MACD",
            "MACDh_12_26_9": "Histogram",
            "MACDs_12_26_9": "Signal",
        }
    )

    # Join indicator DataFrames
    df = df.join([bbands, macd])

    return df

In [34]:
enriched_df = calculate_technical_indicators(asset_df_with_AR)
enriched_df.head()

index         Open         High          Low  \
Date       Asset                                                    
2023-01-01 ADA-USD       0     0.246414     0.249771     0.244245   
           ETH-USD       0  1196.713623  1203.475342  1192.885376   
           LINK-USD      0     5.568981     5.628169     5.517975   
           LTC-USD       0    70.002823    71.328911    69.234886   
           SOL-USD       0     9.961036    10.052801     9.721011   

                           Close      Volume  Return  Expected Return  EMA_50  \
Date       Asset                                                                
2023-01-01 ADA-USD      0.249771   113369989     NaN              NaN     NaN   
           ETH-USD   1200.964844  2399674550     NaN              NaN     NaN   
           LINK-USD     5.622443   109175362     NaN              NaN     NaN   
           LTC-USD     70.815659   344383885     NaN              NaN     NaN   
           SOL-USD      9.982173   194221164     NaN              NaN     NaN   

                     EMA_200  RSI  ATR  Lower_Band  Middle_Band  Upper_Band  \
Date       Asset                                                              
2023-01-01 ADA-USD       NaN  NaN  NaN         NaN          NaN         NaN   
           ETH-USD       NaN  NaN  NaN         NaN          NaN         NaN   
           LINK-USD      NaN  NaN  NaN         NaN          NaN         NaN   
           LTC-USD       NaN  NaN  NaN         NaN          NaN         NaN   
           SOL-USD       NaN  NaN  NaN         NaN          NaN         NaN   

                     Band_Width  Percent_B  MACD  Histogram  Signal  
Date       Asset                                                     
2023-01-01 ADA-USD          NaN        NaN   NaN        NaN     NaN  
           ETH-USD          NaN        NaN   NaN        NaN     NaN  
           LINK-USD         NaN        NaN   NaN        NaN     NaN  
           LTC-USD          NaN        NaN   NaN        NaN     NaN  
           SOL-USD          NaN        NaN   NaN        NaN     NaN

Third function to add signal from strategies 

In [35]:
def load_data_and_apply_strategies(df_all, strategy_map):
    df_list = []

    for asset_name in strategy_map.index:
        try:
            # Get strategy module + class
            strat_class_name = strategy_map.loc[asset_name, "Strategy"]
            strat_file_name = classname_to_filename[strat_class_name]

            strategy_module = importlib.import_module(
                f"strategies.custom.{strat_file_name}"
            )
            strategy_class = getattr(strategy_module, strat_class_name)
            strategy_instance = strategy_class()

            # Get asset's DataFrame slice
            asset_df = df_all.xs(asset_name, level="Asset").copy()

            # Apply strategy
            asset_df = strategy_instance.generate_signals(asset_df)

            # Restore asset label if removed
            asset_df["Asset"] = asset_name
            df_list.append(asset_df.reset_index())

        except Exception as e:
            print(f"Error processing {asset_name} ({strat_class_name}): {e}")
            continue

    # Combine and restore multi-index
    df_with_signals = pd.concat(df_list).set_index(["Date", "Asset"]).sort_index()
    return df_with_signals

In [36]:
df_with_signals = load_data_and_apply_strategies(enriched_df, strategy_map_new)

Error processing AES (Momentum): 'Momentum' object is not subscriptable


c:\Users\LENOVO\Desktop\AY2425 Sem2\DSE4211\cryptoto\src\strategies\custom\bollinger_bands_breakout.py:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if df["Close"][pos] > bbands["Upper_Band_200"][pos]:
c:\Users\LENOVO\Desktop\AY2425 Sem2\DSE4211\cryptoto\src\strategies\custom\bollinger_bands_breakout.py:37: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  elif df["Close"][pos] < bbands["Lower_Band_200"][pos]:
c:\Users\LENOVO\Desktop\AY2425 Sem2\DSE4211\cryptoto\src\strategies\custom\bollinger_bands_breakout.py:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, intege

Error processing LRCX (Momentum): 'Momentum' object is not subscriptable
Error processing MOH (Momentum): 'Momentum' object is not subscriptable
Error processing PLD (Momentum): 'Momentum' object is not subscriptable


c:\Users\LENOVO\Desktop\AY2425 Sem2\DSE4211\cryptoto\src\strategies\custom\bollinger_bands_breakout.py:35: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if df["Close"][pos] > bbands["Upper_Band_200"][pos]:
c:\Users\LENOVO\Desktop\AY2425 Sem2\DSE4211\cryptoto\src\strategies\custom\bollinger_bands_breakout.py:37: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  elif df["Close"][pos] < bbands["Lower_Band_200"][pos]:


In [37]:
valid_df_with_signals = df_with_signals.loc[df_with_signals.index.get_level_values('Date') >= '2025-01-01']
valid_df_with_signals

index         Open         High          Low  \
Date       Asset                                                    
2025-01-01 ADA-USD     731     0.843766     0.929422     0.838926   
           ETH-USD     731  3332.406494  3366.531494  3310.255859   
           LINK-USD    731    20.001163    21.771200    19.707378   
           LTC-USD     731   103.068642   107.728760   102.086693   
           SOL-USD     731   189.266922   194.818497   187.879456   
...                    ...          ...          ...          ...   
2025-02-28 SOL-USD     789   137.620773   148.183838   125.742218   
           STZ         540   174.970001   176.699997   173.029999   
           TSLA        540   279.500000   293.880005   273.600006   
           UNI-USD     789     0.000170     0.000170     0.000158   
           XRP-USD     789     2.198640     2.198640     1.957408   

                           Close       Volume    Return  Expected Return  \
Date       Asset                                                           
2025-01-01 ADA-USD      0.917592    926095054  0.087497              NaN   
           ETH-USD   3353.504150  14195410493  0.006293              NaN   
           LINK-USD    21.674845    670017230  0.083668         0.001535   
           LTC-USD    104.812088    532072625  0.016917              NaN   
           SOL-USD    193.873734   2324231668  0.024354        -0.000526   
...                          ...          ...       ...              ...   
2025-02-28 SOL-USD    148.030014   7544673033  0.075637        -0.004608   
           STZ        175.500000      2913300  0.013045              NaN   
           TSLA       292.980011    115697000  0.039120              NaN   
           UNI-USD      0.000170            3  0.000000              NaN   
           XRP-USD      2.145595   8544047699 -0.024130              NaN   

                         EMA_50     EMA_200  ...   Upper_Band  Band_Width  \
Date       Asset                             ...                            
2025-01-01 ADA-USD   243.346506  287.770892  ...   553.548394  433.836905   
           ETH-USD   365.313472  318.275701  ...  1772.400709  843.557455   
           LINK-USD  351.837448  315.324448  ...  1747.307174  919.070253   
           LTC-USD   102.655285   85.069266  ...  1748.565632  904.881276   
           SOL-USD   336.335416  312.042176  ...  1722.245358  964.682017   
...                         ...         ...  ...          ...         ...   
2025-02-28 SOL-USD   223.909769  237.890667  ...  1248.717906  687.258019   
           STZ       222.011347  237.269864  ...  1249.537845  682.579215   
           TSLA      224.794432  237.824194  ...  1255.700065  654.239624   
           UNI-USD   215.978971  235.457786  ...  1255.110911  674.567172   
           XRP-USD   207.593348  233.136271  ...  1255.058603  687.866832   

                     Percent_B        MACD   Histogram     Signal  \
Date       Asset                                                    
2025-01-01 ADA-USD    0.270710  -61.778801  -23.839428 -37.939373   
           ETH-USD    1.551785  198.878673  189.454437   9.424236   
           LINK-USD   0.398746  135.044116  100.495904  34.548212   
           LTC-USD    0.426083   90.124353   44.460912  45.663440   
           SOL-USD    0.464293   61.008380   12.275952  48.732428   
...                        ...         ...         ...        ...   
2025-02-28 SOL-USD    0.431016  -29.742491  -23.624042  -6.118449   
           STZ        0.444299  -28.503398  -17.907960 -10.595439   
           TSLA       0.499474  -17.836150   -5.792569 -12.043581   
           UNI-USD    0.351757  -32.646969  -16.482710 -16.164258   
           XRP-USD    0.355726  -43.707697  -22.034751 -21.672946   

                     TotalSignal  level_0  EMASignal  AvgVolume10  
Date       Asset                                                   
2025-01-01 ADA-USD             0      NaN        NaN          NaN  
           ETH-USD            

In [38]:
def calculate_fee(amount):
    return max(0.00025 * amount, 0.50)

In [39]:
#With TP and SL and NO CASH FLOOR
def auto_trade_TP_SL(df, strategy_map, config):
    if not isinstance(df.index, pd.MultiIndex):
        df = df.set_index(["date", "asset"])

    assets = df.index.get_level_values(1).unique()
    print(assets)
    dates = df.index.get_level_values(0).unique()

    # Initialize portfolio and cash
    portfolio = {asset: {"units": 0, "entry_price": None} for asset in assets}
    cash = config["starting_cash"]
    trade_log = []
    daily_values = []

    for date in dates:
        # Calculate total portfolio value at start of day
        total_value = cash + sum(
            portfolio[asset]["units"] * df.loc[(date, asset), "Close"]
            for asset in assets
            if (date, asset) in df.index
        )

        # Calculate thresholds
        #cash_floor = config["cash_floor_pct"] * total_value
        # cash_ceiling = config['cash_ceiling_pct'] * total_value

        for asset in assets:
            if (date, asset) not in df.index:
                continue
            row = df.loc[(date, asset)]
            current_price = row["Close"]
            signal = row["TotalSignal"]
            # asset_type = strategy_map.loc[asset, 'type']
            # asset_type = strategy_map.loc[strategy_map['asset'] == asset, 'type'].values[0]

            # Get max allocation for this asset
            ideal_proportion = strategy_map.loc[asset, "Weight"]
            # ideal_proportion = strategy_map.loc[strategy_map['asset'] == asset, 'weight'].values[0]
            max_asset_value = ideal_proportion * total_value
            current_allocation = portfolio[asset]["units"] * current_price
            remaining_allocation = max_asset_value - current_allocation
            buy_chunk = config["buy_pct"] * max_asset_value
            buy_amount = min(buy_chunk, remaining_allocation)

            # ---------- SELL CONDITIONS ----------
            # TP/SL check
            
            if portfolio[asset]["units"] > 0:
                entry_price = portfolio[asset]["entry_price"]
                profit_pct = (current_price - entry_price) / entry_price
                if (
                    profit_pct >= config["tp_pct"] or profit_pct <= -config["sl_pct"]
                ):  # IMPLEMENT DYNAMIC TP AND SL
                    if asset.endswith("-USD"):  # Crypto
                        sell_price = current_price * 0.99  # apply 1% spread
                        proceeds = portfolio[asset]["units"] * sell_price
                    else:
                        fee = calculate_fee(portfolio[asset]["units"] * current_price)
                        proceeds = portfolio[asset]["units"] * current_price - fee
                    cash += proceeds
                    trade_log.append(
                        {
                            "asset": asset,
                            "date": date,
                            "action": (
                                "SELL_TP"
                                if profit_pct >= config["tp_pct"]
                                else "SELL_SL"
                            ),
                            "price": current_price,
                            "units": portfolio[asset]["units"],
                        }
                    )
                    portfolio[asset] = {"units": 0, "entry_price": None}
                    continue  # skip further actions after selling

            # Strategy signal: SHORT → sell everything
            if signal == 1 and portfolio[asset]["units"] > 0:
                if asset.endswith("-USD"):  # Crypto
                    sell_price = current_price * 0.99  # apply 1% spread
                    proceeds = portfolio[asset]["units"] * sell_price
                else:
                    fee = calculate_fee(portfolio[asset]["units"] * current_price)
                    proceeds = portfolio[asset]["units"] * current_price - fee
                cash += proceeds
                trade_log.append(
                    {
                        "asset": asset,
                        "date": date,
                        "action": "SELL_SHORT",
                        "price": current_price,
                        "units": portfolio[asset]["units"],
                    }
                )
                portfolio[asset] = {"units": 0, "entry_price": None}
                continue

            # ---------- BUY CONDITIONS ----------
            if signal == 2:
                expected_return = row.get("Expected Return", None)
                baseline_fee_pct = 0.00025
                if expected_return is None or expected_return <= 2 * baseline_fee_pct:
                    continue  # don't buy if expected return is missing or too low

                # Check if enough cash remains after purchase
                if buy_amount > 0:
                    if asset.endswith("-USD"):  # Crypto
                        buy_price = current_price * 1.01  # apply 1% spread
                        new_units = buy_amount / buy_price
                    else:
                        fee = calculate_fee(buy_amount)
                        new_units = (buy_amount - fee) / current_price

                    old_units = portfolio[asset]["units"]
                    old_price = portfolio[asset]["entry_price"]

                    total_units = old_units + new_units
                    new_avg_price = (
                        (old_units * old_price + new_units * current_price)
                        / total_units
                        if old_units > 0
                        else current_price
                    )  # update price in portfolio so that we can calculate the profit/ loss accurately

                    # Update portfolio
                    portfolio[asset]["units"] = total_units
                    portfolio[asset]["entry_price"] = new_avg_price
                    cash -= buy_amount

                    # Log trade
                    trade_log.append(
                        {
                            "asset": asset,
                            "date": date,
                            "action": "BUY_ADD" if old_units > 0 else "BUY_NEW",
                            "price": current_price,
                            "units": new_units,
                        }
                    )
        eod_value = cash + sum(
            portfolio[asset]['units'] * df.loc[(date, asset), 'Close']
            for asset in assets if (date, asset) in df.index
        )
        # Build daily row with total value and each asset's value
        daily_row = {'date': date, 'portfolio_value': eod_value}
        for asset in assets:
            if (date, asset) in df.index:
                asset_price = df.loc[(date, asset), 'Close']
                asset_units = portfolio[asset]['units']
                daily_row[asset] = asset_units * asset_price
            else:
                daily_row[asset] = 0  # no data that day

        daily_values.append(daily_row)

    # ---------- FINAL SUMMARY ----------
    final_value = cash + sum(
        portfolio[asset]["units"] * df.loc[(dates[-1], asset), "Close"]
        for asset in assets
        if (dates[-1], asset) in df.index
    )

    total_profit = final_value - config["starting_cash"]

    print("\nFinal Portfolio Value:", round(final_value, 2))
    print("Total Profit:", round(total_profit, 2))
    print("Number of Trades:", len(trade_log))

    trade_df = pd.DataFrame(trade_log)

    if not trade_df.empty:
        # Add a signed P&L for each trade
        trade_df["signed_value"] = trade_df["price"] * trade_df["units"]
        trade_df["signed_value"] = trade_df.apply(
            lambda row: (
                -row["signed_value"]
                if row["action"].startswith("BUY")
                else row["signed_value"]
            ),
            axis=1,
        )

        # Net P&L by asset
        pnl_per_asset = trade_df.groupby("asset")["signed_value"].sum().sort_values()

        print("\nNet P&L by Asset (lowest to highest):")
        print(pnl_per_asset)

        worst_asset = pnl_per_asset.idxmin()
        worst_loss = pnl_per_asset.min()

        print(
            f"\nAsset that pulled down profit the most: {worst_asset} (${round(worst_loss, 2)})"
        )

    else:
        print("\n(No trades were executed during the period.)")
    
    pd.DataFrame(daily_values).to_csv("../data/misc/daily_portfolio_value.csv", index=False)
    print("Daily portfolio value saved to 'daily_portfolio_value.csv'")

    trade_df.to_csv("../data/misc/trade_log.csv", index=False)
    print("Trade log saved to 'trade_log.csv'")

    return trade_df, final_value

In [40]:
auto_trade_TP_SL(valid_df_with_signals, strategy_map_new, config)

Index(['ADA-USD', 'ETH-USD', 'LINK-USD', 'LTC-USD', 'SOL-USD', 'UNI-USD',
       'XRP-USD', 'ADI', 'AKAM', 'BLDR', 'CARR', 'CBRE', 'CSGP', 'DECK',
       'DXCM', 'GM', 'GS', 'INTC', 'LII', 'MCHP', 'MCO', 'ODFL', 'PCG', 'PHM',
       'STZ', 'TSLA'],
      dtype='object', name='Asset')

Final Portfolio Value: 1030994.17
Total Profit: 30994.17
Number of Trades: 42

Net P&L by Asset (lowest to highest):
asset
PHM        -33773.151265
GM         -32613.050942
PCG        -30382.733791
LII        -25942.231341
LINK-USD      274.709919
XRP-USD      1365.618710
DECK         3225.297605
SOL-USD      3352.186454
MCHP         4013.910959
INTC         7296.927019
ODFL        10363.497017
Name: signed_value, dtype: float64

Asset that pulled down profit the most: PHM ($-33773.15)
Daily portfolio value saved to 'daily_portfolio_value.csv'
Trade log saved to 'trade_log.csv'


(       asset        date      action       price        units  signed_value
 0        LII  2025-01-02     BUY_NEW  604.520020    50.447337 -30496.424882
 1       ODFL  2025-01-02     BUY_NEW  175.448822   310.148503 -54415.189495
 2        PHM  2025-01-02     BUY_NEW  107.412056   317.828386 -34138.600402
 3    SOL-USD  2025-01-05     BUY_NEW  213.393631   172.016167 -36707.154532
 4    SOL-USD  2025-01-07     SELL_SL  202.230255   172.016167  34786.873403
 5    XRP-USD  2025-01-07     BUY_NEW    2.272350  8123.077703 -18458.476211
 6        PHM  2025-01-08  SELL_SHORT  110.226082   317.828386  35032.977706
 7    XRP-USD  2025-01-11     SELL_TP    2.576770  8123.077703  20931.303479
 8   LINK-USD  2025-01-13     BUY_NEW   19.361107   961.748986 -18620.524904
 9    SOL-USD  2025-01-13     BUY_NEW  182.835358   228.482846 -41774.742828
 10      MCHP  2025-01-13     BUY_NEW   55.493446   659.953273 -36623.081542
 11  LINK-USD  2025-01-15     SELL_TP   22.114241   961.748986  21268.348519

In [41]:
#Without TP and SL
def auto_trade_no_TP_SL(df, strategy_map, config):
    if not isinstance(df.index, pd.MultiIndex):
        df = df.set_index(["date", "Asset"])

    assets = df.index.get_level_values(1).unique()
    print(assets)
    dates = df.index.get_level_values(0).unique()

    # Initialize portfolio and cash
    portfolio = {asset: {"units": 0, "entry_price": None} for asset in assets}
    cash = config["starting_cash"]
    trade_log = []
    daily_values = []

    for date in dates:
        # Calculate total portfolio value at start of day
        total_value = cash + sum(
            portfolio[asset]["units"] * df.loc[(date, asset), "Close"]
            for asset in assets
            if (date, asset) in df.index
        )

        # Calculate thresholds
        #cash_floor = config["cash_floor_pct"] * total_value
        # cash_ceiling = config['cash_ceiling_pct'] * total_value

        for asset in assets:
            if (date, asset) not in df.index:
                continue
            row = df.loc[(date, asset)]
            current_price = row["Close"]
            signal = row["TotalSignal"]
            # asset_type = strategy_map.loc[asset, 'type']
            # asset_type = strategy_map.loc[strategy_map['asset'] == asset, 'type'].values[0]

            # Get max allocation for this asset
            ideal_proportion = strategy_map.loc[asset, "Weight"]
            # ideal_proportion = strategy_map.loc[strategy_map['asset'] == asset, 'weight'].values[0]
            max_asset_value = ideal_proportion * total_value
            current_allocation = portfolio[asset]["units"] * current_price
            remaining_allocation = max_asset_value - current_allocation
            buy_chunk = config["buy_pct"] * max_asset_value
            buy_amount = min(buy_chunk, remaining_allocation)

            # ---------- SELL CONDITIONS ----------
            # Strategy signal: SHORT → sell everything
            if signal == 1 and portfolio[asset]["units"] > 0:
                if asset.endswith("-USD"):  # Crypto
                    sell_price = current_price * 0.99  # apply 1% spread
                    proceeds = portfolio[asset]["units"] * sell_price
                else:
                    fee = calculate_fee(portfolio[asset]["units"] * current_price)
                    proceeds = portfolio[asset]["units"] * current_price - fee
                cash += proceeds
                trade_log.append(
                    {
                        "asset": asset,
                        "date": date,
                        "action": "SELL_SHORT",
                        "price": current_price,
                        "units": portfolio[asset]["units"],
                    }
                )
                portfolio[asset] = {"units": 0, "entry_price": None}
                continue

            # ---------- BUY CONDITIONS ----------
            if signal == 2:
                expected_return = row.get("Expected Return", None)
                baseline_fee_pct = 0.00025
                if expected_return is None or expected_return <= 2 * baseline_fee_pct:
                    continue  # don't buy if expected return is missing or too low

                # Check if enough cash remains after purchase
                if buy_amount > 0:
                    if asset.endswith("-USD"):  # Crypto
                        buy_price = current_price * 1.01  # apply 1% spread
                        new_units = buy_amount / buy_price
                    else:
                        fee = calculate_fee(buy_amount)
                        new_units = (buy_amount - fee) / current_price

                    old_units = portfolio[asset]["units"]
                    old_price = portfolio[asset]["entry_price"]

                    total_units = old_units + new_units
                    new_avg_price = (
                        (old_units * old_price + new_units * current_price)
                        / total_units
                        if old_units > 0
                        else current_price
                    )  # update price in portfolio so that we can calculate the profit/ loss accurately

                    # Update portfolio
                    portfolio[asset]["units"] = total_units
                    portfolio[asset]["entry_price"] = new_avg_price
                    cash -= buy_amount

                    # Log trade
                    trade_log.append(
                        {
                            "asset": asset,
                            "date": date,
                            "action": "BUY_ADD" if old_units > 0 else "BUY_NEW",
                            "price": current_price,
                            "units": new_units,
                        }
                    )
        eod_value = cash + sum(
            portfolio[asset]['units'] * df.loc[(date, asset), 'Close']
            for asset in assets if (date, asset) in df.index
        )
        # Build daily row with total value and each asset's value
        daily_row = {'date': date, 'portfolio_value': eod_value}
        for asset in assets:
            if (date, asset) in df.index:
                asset_price = df.loc[(date, asset), 'Close']
                asset_units = portfolio[asset]['units']
                daily_row[asset] = asset_units * asset_price
            else:
                daily_row[asset] = 0  # no data that day

        daily_values.append(daily_row)

    # ---------- FINAL SUMMARY ----------
    final_value = cash + sum(
        portfolio[asset]["units"] * df.loc[(dates[-1], asset), "Close"]
        for asset in assets
        if (dates[-1], asset) in df.index
    )

    total_profit = final_value - config["starting_cash"]

    print("\nFinal Portfolio Value:", round(final_value, 2))
    print("Total Profit:", round(total_profit, 2))
    print("Number of Trades:", len(trade_log))

    trade_df = pd.DataFrame(trade_log)

    if not trade_df.empty:
        # Add a signed P&L for each trade
        trade_df["signed_value"] = trade_df["price"] * trade_df["units"]
        trade_df["signed_value"] = trade_df.apply(
            lambda row: (
                -row["signed_value"]
                if row["action"].startswith("BUY")
                else row["signed_value"]
            ),
            axis=1,
        )

        # Net P&L by asset
        pnl_per_asset = trade_df.groupby("asset")["signed_value"].sum().sort_values()

        print("\nNet P&L by Asset (lowest to highest):")
        print(pnl_per_asset)

        worst_asset = pnl_per_asset.idxmin()
        worst_loss = pnl_per_asset.min()

        print(
            f"\nAsset that pulled down profit the most: {worst_asset} (${round(worst_loss, 2)})"
        )

    else:
        print("\n(No trades were executed during the period.)")
    
    pd.DataFrame(daily_values).to_csv("../data/misc/daily_portfolio_value_no_TP_SL.csv", index=False)
    print("Daily portfolio value saved to 'daily_portfolio_value_no_TP_SL.csv'")

    trade_df.to_csv("../data/misc/trade_log_no_TP_SL.csv", index=False)
    print("Trade log saved to 'trade_log_no_TP_SL.csv'")

    return trade_df, final_value

In [ ]:
auto_trade_no_TP_SL(valid_df_with_signals, strategy_map_new, config)